In [ ]:
import json, os
from primeqa.ir.scripts.elastic_ingestion import process_url
working_dir="/home/raduf/sandbox2/primeqa/benchmark/sap"

In [69]:
def openf(file, mode="r"):
    return open(os.path.join(working_dir, file), mode)

In [ ]:
loiomap = {}
with open(os.path.join(working_dir,"loio2productId.map")) as inp:
    for line in inp:
        a = line.strip().split(" ")
        loiomap[a[0]] = a[1]

In [45]:
def replace_enter(s: str)->str:
    if s.find("\n") >= 0 and s.find("\\n") < 0:
        return s.replace("\n", "\\n")
    else:
        return s


In [46]:
import csv
with open(os.path.join(working_dir,"benchmark_v5.csv")) as inp,\
        open(os.path.join(working_dir,"benchmark_v5.fixed.csv"), "w") as out:
    csvreader = csv.reader(inp, quotechar='"', delimiter=",")
    csvwriter = csv.writer(out, quotechar='"', delimiter=",")
    for row in csvreader:
        res = ""
        prods = {}
        for loio in row[5:8]:
            if loio != "":
                l = loio.replace("loio", "")
                if l in loiomap:
                    prods[loiomap[l]] = 1
        if len(prods)>0:
            row[3] = " ".join([f"{s}" for s in prods.keys()])
        if row[4].find("\n")>=0:
            questions = [s for s in row[4].split("\n") if s!=""]
            for q in questions:
                row[4] = q
                row[2] = replace_enter(row[2])
                csvwriter.writerow(row)
        else:
            row[2] = replace_enter(row[2])
            csvwriter.writerow(row)
            

In [58]:
with open(os.path.join(working_dir,"benchmark_v5_fixed.csv")) as inp,\
        open(os.path.join(working_dir,"benchmark_v5_fixed_filtered.csv"), "w") as out:
    csvreader = csv.reader(inp, quotechar='"', delimiter=",")
    csvwriter = csv.writer(out, quotechar='"', delimiter=",")
    n=0
    for row in csvreader:
        if row[5] != "":
            csvwriter.writerow(row)
            n+=1
    print(f"{n} rows processed")
        

611 rows processed


In [74]:
# lang="pt"
for lang in ['pt', 'de', 'es']:
    loios={s.strip():1 for s in openf(f"{lang}.loios").readlines()}
    with openf(f"sap-{lang}.jsonl") as inp, \
            openf(f"sap-{lang}-gold.jsonl", "w") as out:
        for line in inp:
            d=json.loads(line)
            url, parts=process_url(d['document_url'], data_type="sap")
            if parts[-1] in loios:
                out.write(line)
